In [44]:
import sys

In [1]:
from langchain_community.llms import CTransformers

In [2]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceHubEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain import ConversationChain
from langchain.chains import ConversationalRetrievalChain

In [3]:
import faiss

In [4]:
loader = CSVLoader(file_path=r'D:\llm-dict\Data\2019.csv')
data = loader.load()

In [5]:
#Split the text inot the chunks 
text_spliter = RecursiveCharacterTextSplitter(chunk_size = 500 , chunk_overlap = 20)

In [6]:
text_chunks = text_spliter.split_documents(data)

In [7]:
embedding = HuggingFaceHubEmbeddings(model = "sentence-transformers/all-MiniLM-L6-v2")

d:\llm-dict\dict_llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
similarity_match = FAISS.from_documents(text_chunks, embedding)

In [9]:
similarity_match.save_local("D:\llm-dict\Data")

In [10]:
docs = similarity_match.similarity_search(query="what is value of GDP percapita of finland provded in the data?")

print(docs)

[Document(page_content='Overall rank: 1\nCountry or region: Finland\nScore: 7.769\nGDP per capita: 1.340\nSocial support: 1.587\nHealthy life expectancy: 0.986\nFreedom to make life choices: 0.596\nGenerosity: 0.153\nPerceptions of corruption: 0.393', metadata={'source': 'D:\\llm-dict\\Data\\2019.csv', 'row': 0}), Document(page_content='Overall rank: 7\nCountry or region: Sweden\nScore: 7.343\nGDP per capita: 1.387\nSocial support: 1.487\nHealthy life expectancy: 1.009\nFreedom to make life choices: 0.574\nGenerosity: 0.267\nPerceptions of corruption: 0.373', metadata={'source': 'D:\\llm-dict\\Data\\2019.csv', 'row': 6}), Document(page_content='Overall rank: 53\nCountry or region: Latvia\nScore: 5.940\nGDP per capita: 1.187\nSocial support: 1.465\nHealthy life expectancy: 0.812\nFreedom to make life choices: 0.264\nGenerosity: 0.075\nPerceptions of corruption: 0.064', metadata={'source': 'D:\\llm-dict\\Data\\2019.csv', 'row': 52}), Document(page_content='Overall rank: 55\nCountry or re

In [11]:
llm = CTransformers(model=r"C:\Users\ganes\Downloads\llama-2-7b-chat.ggmlv3.q4_0.bin", model_type="llama",max_new_token = 512, tempreture = 0.5)

In [12]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever = similarity_match.as_retriever())

In [ ]:

chat_history = []

query = input(f"input prompt .....")

if query == "exit":
    print("Exiting")
    sys.exit()


result = qa({"question": query, "chat_history":chat_history})
print("Response: ", result["answer"])